In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames[:5]:
#         print(os.path.join(dirname, filename))

# /kaggle/input/fast-food-classification-dataset/Fast Food Data/Training Data/Pizza/Pizza-Train (907).jpeg

# import datasets use keras 

In [ ]:
import glob 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import numpy as np
import pandas as pd 



In [ ]:
train_data_dir ='/kaggle/input/fast-food-classification-dataset/Fast Food Data/Training Data' 
val_data_dir = '/kaggle/input/fast-food-classification-dataset/Fast Food Data/Validation Data'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    )
val_datagen = ImageDataGenerator(
    rescale=1./255,
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (180,180),
    batch_size=32,
    # class_mode default value is clasification
)
val_generator = train_datagen.flow_from_directory(
    val_data_dir,
    target_size = (180,180),
    batch_size=32,
    
)

In [ ]:
labels = ['Burger','Donut','Hot Dog','Pizza','Sandwich']

# build the model 

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet',include_top=False)


In [ ]:
# from keras.layers.serialization import activation
result = model.output 
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
result = GlobalAveragePooling2D()(result)
result = Dense(512,activation='relu')(result)
predictions = Dense(5,activation='sigmoid')(result)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Model
resnet_model = Model(inputs=model.input,outputs=predictions)

resnet_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(
#     model,to_file = 'vgg_model.png'
# )

In [ ]:
with tf.device("/device:GPU:0"):
    
    history = resnet_model.fit_generator(
        train_generator,
        epochs=50,
        shuffle=True,
        verbose=1,
        validation_data=val_generator)

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained'], loc='upper left')
plt.show()

In [ ]:
history.history['accuracy'][-1]